In [ ]:
!pip install aladdinsdk
!pip install asdk-plugin-analytics
!pip install asdk-plugin-compliance
!pip install asdk-plugin-portfolio
!pip list | grep sdk

# Initial configuration setup

### Setup a configuration file.

- Open a terminal and run `aladdinsdk-cli`, and follow the steps to create a configuration file.
- Per the final instruction in the CLI, set the ASDK_USER_CONFIG_FILE environment variable **before** importing aladdinsdk

In [ ]:
import os
os.environ["ASDK_USER_CONFIG_FILE"] = '<path to config file>'

# Set environment variable prior to import, either in code / terminal
import aladdinsdk

Utility to validate configuration is picked correctly:

In [ ]:
from aladdinsdk.config import print_current_user_config
print_current_user_config()

In [ ]:
# Setup formatting functions for later use

def color_text(c, t):
    ct = f"\033[{c}{t}\033[00m"
    return ct
GREEN = '32m'

def bold_text(t):
    bt = f"\033[1m{t}\033[0m"
    return bt

------------

# API client

List currently supported APIs:

In [ ]:
from aladdinsdk.api import get_api_names

for api in get_api_names():
    print("--> " + api)

## Create an instance of API client

Example workflow: Bulk Retrieve and Expand on Compliance Rule Assignments for Portfolios in Portfolio Groups.



In [ ]:
from aladdinsdk.api import AladdinAPI

api_instance_port_group = AladdinAPI("PortfolioGroupAPI")
api_instance_compl_assignments = AladdinAPI("ComplianceRuleAssignmentAPI")

API introspection:

In [ ]:
api_instance_compl_assignments.get_api_endpoint_path_tuples()

#### List portfolios under a given Portfolio Group

_Sample GET call_

In [ ]:
port_group_members = api_instance_port_group.get("/portfolioGroups/{id}/members", id="TEST-FUNDS")

port_group_members

#### Understand Compliance Rule Assignments for given Portfolio Group member

_Sample POST request_

In [ ]:
for node in port_group_members.nodes:
    body = { 
        "query":{
            "portfolioName": node.portfolio_ticker
        }
    }
    resp = api_instance_compl_assignments.post("/complianceRuleAssignments:filter", body)
    
    if len(resp.compliance_rule_assignments) == 0:
        print(f'\tNo compliance rules for portfolio "{node.portfolio_ticker}"')
    else:
        print(color_text(GREEN,bold_text(f'Portfolio "{node.portfolio_ticker}" has {len(resp.compliance_rule_assignments)} compliance rules')))

### Simplified API call example

In [ ]:
esg_data_api_instance = AladdinAPI("EsgDataAPI")

esg_data_response = esg_data_api_instance.get('/metadata:retrieve', provider_id="MSCI")

print(f"id: {esg_data_response.provider_id}, category: {esg_data_response.provider_category}, metadata length: {len(esg_data_response.esg_metadata)}")

----- 

# ADC Connectivity


In [ ]:
import aladdinsdk
from aladdinsdk.adc.client import ADCClient

adc_client = ADCClient()

df = adc_client.query_sql('SELECT * FROM CASH_ENTRY_V2 LIMIT 10')
df

**Note:**
- Configuration files are optional, each config can be overriden in-line
- Environment variables take precedence, and may be used to set unmodifiable configurations

-----

# Common Dev Utilities

## Data transformation - JSON to dataframe

In [ ]:
from aladdinsdk.api import AladdinAPI
train_journey_api_instance = AladdinAPI("TrainJourneyAPI")
req_body_json = { "query": { "departingStationId": "TS_1441" } }
train_journey_response = train_journey_api_instance.post('/trainJourneys:filter', req_body_json)

In [ ]:
train_journey_response.json()

In [ ]:
from aladdinsdk.common.datatransformation import json_to_pandas

json_to_pandas.convert(train_journey_response.json(), "train_journeys.[*]")

## Batch Processing

In [ ]:
import aladdinsdk
from aladdinsdk.common.batch.action import SDKActionBuffer, SDKAction
from aladdinsdk.api.client import AladdinAPI

action_buffer = SDKActionBuffer()

api_instance_train_journey = AladdinAPI("TrainJourneyAPI")
req_body_json = { "query": { "departingStationId": "TS_1441" } }

api_post_action = SDKAction(api_instance_train_journey.post, "/trainJourneys:filter", req_body_json)

api_get_action = SDKAction(api_instance_train_journey.get, '/trainJourneys/{id}', id='TJ_4')

action_buffer.append_action(api_post_action)
action_buffer.append_action(api_get_action)

# action_buffer.run_sequential()
action_buffer.run_parallel()

response_map = action_buffer.get_response_map()
for action_uid in response_map:
    res = response_map[action_uid]
    print(f"result for action {action_uid}: {type(res)}")

In [ ]:
# Batch Processing example with print statements to understand the flow

from aladdinsdk.common.batch.action import SDKActionBuffer, SDKAction
import time

def func1(num, sleep_time):
    from datetime import datetime
    starttime = datetime.now()
    print(f"start {num}-{sleep_time}")
    time.sleep(sleep_time)
    endtime = datetime.now()
    print(f"{endtime - starttime} end {num}-{sleep_time}")
    return num

# Action buffer setup
action_buffer = SDKActionBuffer()
action_buffer.append_action(SDKAction(func1, 1, 2))
action_buffer.append_action(SDKAction(func1, 2, 3))
action_buffer.append_action(SDKAction(func1, 3, 8))
action_buffer.append_action(SDKAction(func1, 4, 8))
action_buffer.append_action(SDKAction(func1, 5, 8))
action_buffer.append_action(SDKAction(func1, 6, 8))
action_buffer.append_action(SDKAction(func1, 7, 8))
action_buffer.append_action(SDKAction(func1, 8, 10))

# action_buffer.run_sequential()  # optional arg: interval: to pause between actions
action_buffer.run_parallel()  # optional arg: max_workers to limit worker threads

# for action in action_buffer.actions:
#     print(f"result for action {action.uid} result {action.result} error {action.error}")
for result in action_buffer.get_response_map().values():
    print(f"result for action {result}")



### Other common utilities:
- Batch Processing
- File exports
- Error handling
- Retry mechanism
- E-mail notifications
- Configuration management

Foundational structure and initial implementations done so far. Additional enhancements are in the pipeline.

Also, inviting all developers to contribute more generic/business-agnostic features to the core SDK via our open source project!

---------------

# Business Abstractions

- API owners / Domain experts can build meaningful business abstractions using these generic utilities.
- The intention is to enable service providers control the user experience at the edge.

In [ ]:
from aladdinsdk.api import AladdinAPI
from aladdinsdk.common.datatransformation import json_to_pandas

def train_journey_summary(relevant_input_param: str):
    """
    - Construct request body
    - Perform validations
    - Invoke API endpoint
    - Validate response
    - Handle errors
    - Transform response
    """
    api_instance_train_journey = AladdinAPI("TrainJourneyAPI")

    req_body_json = {
        "query": {
            "departingStationId": relevant_input_param
        }
    }

    response = api_instance_train_journey.call_api("train_journey_api_filter_train_journeys", req_body_json)

    tf_df = json_to_pandas.convert(response.json(), "train_journeys.[*]")

    tf_df = tf_df[[
        'train_journeys.train_id',
        'train_journeys.journey_date',
        'train_journeys.journey_time',
        'train_journeys.short_code'
    ]]
    return tf_df

## Simplified end user experience

In [ ]:
train_journey_summary('TS_1441')

In [ ]:
train_journey_summary('TS_786')

## DomainSDK Error Handling Mechanism

AladdinSDK simplifies error handling internally by using a decorator (asdk_exception_handler) that intercepts raised exceptions and maps them to specific handlers. This decorator and handler framework is made available to AladdinSDK users and DomainSDK developers.

To utilize this capability:
- Implement an exception handler class by implementing AbstractAsdkExceptionHandler, and configure the newly created enum
- Provide this class definition to the core AladdinSDK by invoking register_handler_class e.g.:
  - e.g.:
    ```py
    from aladdinsdk.common.error import handler
    handler.register_handler_class(DomainExceptionHandler)
    ```
- The decorator asdk_exception_handler will now map any matching exceptions to the registered handlers and invoke the handle_error method.


------------

### References:
- Engineering blog - https://engineering.blackrock.com/open-sourcing-the-aladdinsdk-empower-python-developers-with-a-quantitative-edge-7f63376061e6
- Open source project - https://github.com/blackrock/aladdinsdk
    - https://pypi.org/project/aladdinsdk/
- Internal getting started guide - https://webster.bfm.com/apps/studio/knowledge-center/get-started-with-aladdinsdk-blk